### Extract by mask

In [1]:
import arcpy

# Set up environment
arcpy.env.workspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June ET.gdb"
project = arcpy.mp.ArcGISProject("CURRENT")
map = project.listMaps()[0]  # Assuming you want to work with the first map

# Define the inputs
years = ["2023", "2022", "2021", "2019", "2018"]
input_rasters = ["OpenET_Ensemble_ET_June_2023_Minnesota.tif","OpenET_Ensemble_ET_June_2022_Minnesota.tif","OpenET_Ensemble_ET_June_2021_Minnesota.tif","OpenET_Ensemble_ET_June_2019_Minnesota.tif", "OpenET_Ensemble_ET_June_2018_Minnesota.tif"]
mask_files = ["2023 corn lc.TIF", "2022 corn lc.TIF", "2021 corn lc.TIF", "2019 corn lc.TIF", "2018 corn lc.TIF"]
output_names = ["June2023cornET", "June2022cornET","June2021cornET", "June2019cornET", "June2018cornET"]

# Loop through each year to process and add to the map
for input_raster, mask_file, output_name in zip(input_rasters, mask_files, output_names):
    # Process each raster
    out_raster = arcpy.sa.ExtractByMask(
        in_raster=input_raster,
        in_mask_data=mask_file,
        extraction_area="INSIDE"
    )
    # Save the raster
    out_path = arcpy.env.workspace + "\\" + output_name
    out_raster.save(out_path)
    
    # Add the saved raster to the map with a specified name
    layer = map.addDataFromPath(out_path)
    layer.name = output_name  # Set the layer name

# Save changes to the project
project.save()

In [2]:
import arcpy
import os

# Define your base workspace
base_output_folder = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET"

# Years to process and their corresponding ET map layers
years = ["2023", "2022", "2021", "2019", "2018"]
et_map_layers = {
    "2023": "June2023cornET",
    "2022": "June2022cornET",
    "2021": "June2021cornET",
    "2019": "June2019cornET",
    "2018": "June2018cornET"
}

# Path to the Minnesota county boundaries shapefile
minnesota_county_shapefile = "mn_county_boundaries"

# Get a list of county names from the attribute table of the county shapefile
county_names = [row[0] for row in arcpy.da.SearchCursor(minnesota_county_shapefile, "CTY_Name")]

# Process each year
for year in years:
    # Set up specific year's workspace
    year_output_folder = os.path.join(base_output_folder, f"June{year}ET")
    if not os.path.exists(year_output_folder):
        os.makedirs(year_output_folder)  # Create the directory if it does not exist

    # Reference the layer name of the ET map for the current year in ArcGIS Pro from the dictionary
    et_map_layer = et_map_layers[year]

    # Create an empty dictionary to store county names and their average ET values for this year
    county_et_averages = {}

    # Iterate through each county
    for county_name in county_names:
        try:
            # Define the SQL query to select the current county
            sql_query = f"CTY_Name = '{county_name}'"

            # Make a feature layer for the current county boundary
            arcpy.MakeFeatureLayer_management(minnesota_county_shapefile, "Current_County_Layer", sql_query)

            # Define the output path for the clipped ET raster
            county_et_map = os.path.join(year_output_folder, f"{county_name.replace(' ', '_')}_June_ET_{year}.tif")

            # Clip the ET raster with the county boundary
            arcpy.management.Clip(et_map_layer, "#", county_et_map, "Current_County_Layer", "#", "ClippingGeometry")

            # Check if raster is valid and has data
            if int(arcpy.management.GetRasterProperties(county_et_map, "ROWCOUNT").getOutput(0)) > 0:
                # Calculate statistics for the clipped raster to ensure 'MEAN' can be computed
                arcpy.management.CalculateStatistics(county_et_map)

                # Calculate the mean of the clipped raster
                result = arcpy.GetRasterProperties_management(county_et_map, "MEAN")
                mean_value = float(result.getOutput(0))

                # Store the county name and its average ET value in the dictionary
                county_et_averages[county_name] = mean_value
            else:
                # Assign NA for empty or no data raster
                county_et_averages[county_name] = "NA"
                print(f"No valid data for {county_name} in {year}, assigned 'NA'.")

            print(f"{county_name} June ET map for {year} has been processed and saved to {county_et_map}. Average value: {county_et_averages[county_name]}")
        except Exception as e:
            print(f"Failed to process {county_name} for {year}: {str(e)}")
            county_et_averages[county_name] = "NA"  # Assign NA in case of any errors

    # Print the dictionary containing county names and their average ET values for the year
    print(f"County-wise Average ET Values for June {year}:")
    for county, avg_et in county_et_averages.items():
        print(f"{county}: {avg_et}")


Lake of the Woods June ET map for 2023 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2023ET\Lake_of_the_Woods_June_ET_2023.tif. Average value: 135.0
Kittson June ET map for 2023 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2023ET\Kittson_June_ET_2023.tif. Average value: 104.469387755102
Roseau June ET map for 2023 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2023ET\Roseau_June_ET_2023.tif. Average value: 114.559633027523
Koochiching June ET map for 2023 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2023ET\Koochiching_June_ET_2023.tif. Average value: 123.111111111111
Marshall June ET map for 2023 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2023ET\Marshall_June_ET_2023.tif. Average value: 108.114369501466
St. Louis June ET map for 2023 has been pro

Wright June ET map for 2023 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2023ET\Wright_June_ET_2023.tif. Average value: 125.023622047244
Anoka June ET map for 2023 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2023ET\Anoka_June_ET_2023.tif. Average value: 132.074074074074
Meeker June ET map for 2023 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2023ET\Meeker_June_ET_2023.tif. Average value: 127.631029986962
Lac qui Parle June ET map for 2023 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2023ET\Lac_qui_Parle_June_ET_2023.tif. Average value: 119.159757330637
Washington June ET map for 2023 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2023ET\Washington_June_ET_2023.tif. Average value: 128.553191489362
Hennepin June ET map for 2023 has been processed a


Failed to process Cook for 2023: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Cook for 2023: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Cook for 2023: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Cook for 2023: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Cook for 2023: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Cook for 2023: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Lake for 2023: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Lake for 2023: ERROR 001100: Failed because no sta

Clearwater June ET map for 2022 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2022ET\Clearwater_June_ET_2022.tif. Average value: 120.892857142857
Red Lake June ET map for 2022 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2022ET\Red_Lake_June_ET_2022.tif. Average value: 109.219298245614
Itasca June ET map for 2022 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2022ET\Itasca_June_ET_2022.tif. Average value: 131.777777777778
Norman June ET map for 2022 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2022ET\Norman_June_ET_2022.tif. Average value: 79.9884488448845
Mahnomen June ET map for 2022 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2022ET\Mahnomen_June_ET_2022.tif. Average value: 89.6
Cass June ET map for 2022 has been processed and saved to C:\Users

Yellow Medicine June ET map for 2022 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2022ET\Yellow_Medicine_June_ET_2022.tif. Average value: 88.0921052631579
Dakota June ET map for 2022 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2022ET\Dakota_June_ET_2022.tif. Average value: 109.6875
Renville June ET map for 2022 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2022ET\Renville_June_ET_2022.tif. Average value: 85.147194719472
Scott June ET map for 2022 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2022ET\Scott_June_ET_2022.tif. Average value: 118.721590909091
Sibley June ET map for 2022 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2022ET\Sibley_June_ET_2022.tif. Average value: 98.0242557883131
Redwood June ET map for 2022 has been processed and saved t

...

Stevens June ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2019ET\Stevens_June_ET_2019.tif. Average value: 95.7536231884057
Stearns June ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2019ET\Stearns_June_ET_2019.tif. Average value: 105.172463768116
Pope June ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2019ET\Pope_June_ET_2019.tif. Average value: 100.594744121715
Isanti June ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2019ET\Isanti_June_ET_2019.tif. Average value: 113.666666666667
Chisago June ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2019ET\Chisago_June_ET_2019.tif. Average value: 114.926666666667
Big Stone June ET map for 2019 has been processed and saved to C:\Us

Rock June ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2019ET\Rock_June_ET_2019.tif. Average value: 104.092226613966
Nobles June ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2019ET\Nobles_June_ET_2019.tif. Average value: 104.227182539683
Jackson June ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2019ET\Jackson_June_ET_2019.tif. Average value: 93.8632124352332
Martin June ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2019ET\Martin_June_ET_2019.tif. Average value: 93.9010896898575
Houston June ET map for 2019 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2019ET\Houston_June_ET_2019.tif. Average value: 112.485401459854
Faribault June ET map for 2019 has been processed and saved to C:\User

Kittson June ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2018ET\Kittson_June_ET_2018.tif. Average value: 110.979797979798
Roseau June ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2018ET\Roseau_June_ET_2018.tif. Average value: 112.637931034483
Koochiching June ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2018ET\Koochiching_June_ET_2018.tif. Average value: 116.0
Marshall June ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2018ET\Marshall_June_ET_2018.tif. Average value: 109.477611940299
Failed to process St. Louis for 2018: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Beltrami June ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\J

Anoka June ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2018ET\Anoka_June_ET_2018.tif. Average value: 100.862068965517
Meeker June ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2018ET\Meeker_June_ET_2018.tif. Average value: 117.746612466125
Lac qui Parle June ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2018ET\Lac_qui_Parle_June_ET_2018.tif. Average value: 81.9138321995464
Washington June ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2018ET\Washington_June_ET_2018.tif. Average value: 119.870689655172
Hennepin June ET map for 2018 has been processed and saved to C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\June ET\June2018ET\Hennepin_June_ET_2018.tif. Average value: 113.906976744186
Chippewa June ET map for 2018 has been process


Failed to process Cook for 2018: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Cook for 2018: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Cook for 2018: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Cook for 2018: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Lake for 2018: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Lake for 2018: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Lake for 2018: ERROR 001100: Failed because no statistics are available.
Failed to execute (GetRasterProperties).

Failed to process Lake for 2018: ERROR 001100: Failed because no sta